## Data Generator

Training용 data image에서 bird에 해당하는 부분만 crop하여 classifier의 data generate

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import tempfile
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps
import time
from tensorflow import keras
from keras.models import Model
import pandas as pd
import glob
import os

In [3]:
df = pd.read_csv('species.csv')
species = df['Eng_name']
species

0      seagull
1    wildgoose
2        crane
3         ibis
Name: Eng_name, dtype: object

In [27]:
detector = tf.saved_model.load("./detector").signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [41]:
def run_detector(detector, path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    converted_img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    result = detector(converted_img)
    result = {key:value.numpy() for key, value in result.items()}

    birdbox = []
    for i in range(len(result["detection_boxes"])):
        if result["detection_class_entities"][i].decode("ascii") == "Bird" and result["detection_scores"][i] >= 0.1 :
            ymin, xmin, ymax, xmax = tuple(result["detection_boxes"][i])
            im_height, im_width, c = img.shape
            (left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
            birdbox.append(img[int(top):int(bottom), int(left):int(right)])

    return birdbox

In [74]:
data_path = "./data/image"

for bird in species:
    if not os.path.exists('./data/crop/' + bird):
        os.makedirs('./data/crop/' + bird)
    cnt = 0
    image_paths = glob.glob(data_path + "/" + bird + "/*")
    for image_path in image_paths:
        birdbox = run_detector(detector, image_path)
        for bird_image in birdbox:
            bird_image = tf.keras.preprocessing.image.array_to_img(bird_image)
            bird_image.save("./data/crop/" + bird + "/" + bird + str(cnt) + ".png", "png")
            cnt = cnt + 1